In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Cài đặt 
!pip install xgboost
!pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1
import warnings
warnings.simplefilter('ignore', FutureWarning)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0


In [2]:
import pandas as pd

data_filled = pd.read_csv('/kaggle/input/clean-aws-month/final_data/filled_data_october.csv')

# 1. Xử lý dữ liệu bị thiếu

In [3]:
# df_cleaned = data.copy()
# df_cleaned = df_cleaned.drop(columns=['B04B', 'B05B', 'B06B', 'VSB', 'CIN'])

# df_cleaned = df_cleaned[~((df_cleaned['AWS'] == -np.inf) | (df_cleaned['AWS'].isnull()))]
# df_cleaned = df_cleaned[~((df_cleaned['SLHF'] == 9999) | (df_cleaned['SSHF'] == 9999))]

In [4]:
# data_filled = df_cleaned.fillna(df_cleaned.median(numeric_only=True))

# 2. Chia train test

In [5]:
def split_by_day(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = df['datetime'].dt.date

    # Lấy danh sách ngày duy nhất
    unique_days = sorted(df['date'].unique())
    n_train_days = int(0.8 * len(unique_days))

    # Tách theo ngày
    train_days = unique_days[:n_train_days]
    test_days = unique_days[n_train_days:]

    train_df = df[df['date'].isin(train_days)].copy()
    test_df = df[df['date'].isin(test_days)].copy()

    # In thông tin
    print(f"Số ngày trong train: {len(train_days)}")
    print(f"Số ngày trong test: {len(test_days)}")
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
    print(f"Ngày trong train:\n{np.array(train_days)}")
    print(f"Ngày trong test:\n{np.array(test_days)}")

    # Xóa cột phụ trợ 'date'
    train_df.drop(columns='date', inplace=True)
    test_df.drop(columns='date', inplace=True)

    return train_df, test_df

# Chia lại cho từng tháng
train_df, test_df = split_by_day(data_filled)

Số ngày trong train: 49
Số ngày trong test: 13
Train shape: (262501, 34)
Test shape: (89319, 34)
Ngày trong train:
[datetime.date(2019, 10, 1) datetime.date(2019, 10, 2)
 datetime.date(2019, 10, 3) datetime.date(2019, 10, 4)
 datetime.date(2019, 10, 5) datetime.date(2019, 10, 6)
 datetime.date(2019, 10, 7) datetime.date(2019, 10, 8)
 datetime.date(2019, 10, 9) datetime.date(2019, 10, 10)
 datetime.date(2019, 10, 11) datetime.date(2019, 10, 12)
 datetime.date(2019, 10, 13) datetime.date(2019, 10, 14)
 datetime.date(2019, 10, 15) datetime.date(2019, 10, 16)
 datetime.date(2019, 10, 17) datetime.date(2019, 10, 18)
 datetime.date(2019, 10, 19) datetime.date(2019, 10, 20)
 datetime.date(2019, 10, 21) datetime.date(2019, 10, 22)
 datetime.date(2019, 10, 23) datetime.date(2019, 10, 24)
 datetime.date(2019, 10, 25) datetime.date(2019, 10, 26)
 datetime.date(2019, 10, 27) datetime.date(2019, 10, 28)
 datetime.date(2019, 10, 29) datetime.date(2019, 10, 30)
 datetime.date(2019, 10, 31) datetime.d

In [6]:
X_train = train_df.drop(columns=['AWS', 'datetime', 'row', 'col'])
y_train = (train_df['AWS'] > 0).astype(int)

X_test = test_df.drop(columns=['AWS', 'datetime', 'row', 'col'])
y_test = (test_df['AWS'] > 0).astype(int)

# 3. Chuẩn hóa

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# 4. Chọn feature

In [8]:
top_features =  ['EWSS', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SSHF', 'TCLW', 'TCW', 'U250', 'U850', 'V250', 'V850', 'B11B', 'B14B', 'I4B', 'IRB']

In [9]:
X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

In [10]:
#Performance evaluation
def print_scores(y_true, y_pred):
  print(classification_report(y_true, y_pred))

# 5. Thêm class weight

In [11]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     68556
           1       0.78      0.56      0.65     20763

    accuracy                           0.86     89319
   macro avg       0.83      0.76      0.78     89319
weighted avg       0.85      0.86      0.85     89319



In [12]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled, y_train)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     68556
           1       0.78      0.56      0.65     20763

    accuracy                           0.86     89319
   macro avg       0.83      0.76      0.78     89319
weighted avg       0.85      0.86      0.85     89319



In [13]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, scale_pos_weight= 5.300729681724353)
model_xgb.fit(X_train_scaled, y_train)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.93      0.86      0.89     68556
           1       0.63      0.79      0.70     20763

    accuracy                           0.84     89319
   macro avg       0.78      0.82      0.80     89319
weighted avg       0.86      0.84      0.85     89319



In [14]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, scale_pos_weight= 5.300729681724353)
model_xgb.fit(X_train_selected, y_train)

y_pred = model_xgb.predict(X_test_selected)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.93      0.80      0.86     68556
           1       0.55      0.81      0.66     20763

    accuracy                           0.80     89319
   macro avg       0.74      0.81      0.76     89319
weighted avg       0.85      0.80      0.82     89319



=> Giải pháp:

1. **Giữ mô hình đã tối ưu + threshold = 0.6** để triển khai hoặc đánh giá cuối cùng.
2. Thử thêm:

   * Tối ưu `max_depth`, `gamma`, `min_child_weight`, `subsample` bằng GridSearchCV hoặc Optuna.
   * Dùng **cross-validation phân tầng (StratifiedKFold)** để đánh giá mô hình ổn định hơn.
3. Nếu cần **giải thích mô hình**, có thể dùng SHAP để giải thích đóng góp các feature.

**Tuning XGBoost không chọn đặc trưng với optuna**

In [15]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=5.300729681724353,
    random_state=42
)
model_xgb.fit(X_train_scaled, y_train)


y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_scaled)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")


              precision    recall  f1-score   support

           0       0.95      0.82      0.88     68556
           1       0.60      0.86      0.71     20763

    accuracy                           0.83     89319
   macro avg       0.77      0.84      0.79     89319
weighted avg       0.87      0.83      0.84     89319

Best threshold: 0.6500000000000001, F1: 0.7174967547994807


In [16]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


THRESHOLD = best_thresh
SCALE_POS_WEIGHT =  5.300729681724353 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled, y_train)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)

# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_train)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:19:31,117] A new study created in memory with name: no-name-5c7a6ecb-6ca3-4ef8-94f5-2405af373ff2
[I 2025-05-04 04:19:36,635] Trial 0 finished with value: 0.6970000955383587 and parameters: {'learning_rate': 0.2351520042763188, 'max_depth': 7, 'min_child_weight': 5, 'subsample': 0.6628007684396079, 'colsample_bytree': 0.7265078068790222, 'gamma': 4.9692816226252035}. Best is trial 0 with value: 0.6970000955383587.
[I 2025-05-04 04:19:41,196] Trial 1 finished with value: 0.7081427264409881 and parameters: {'learning_rate': 0.21732824138501913, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.8905935818722354, 'colsample_bytree': 0.9548165307365866, 'gamma': 0.8886619722209055}. Best is trial 1 with value: 0.7081427264409881.
[I 2025-05-04 04:19:45,805] Trial 2 finished with value: 0.6941116056145156 and parameters: {'learning_rate': 0.018045474451335898, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.5999609453948862, 'colsample_bytree': 0.5827381461197314, 

Best trial:
FrozenTrial(number=21, state=1, values=[0.7173410152328576], datetime_start=datetime.datetime(2025, 5, 4, 4, 21, 12, 615724), datetime_complete=datetime.datetime(2025, 5, 4, 4, 21, 16, 227347), params={'learning_rate': 0.1776634149900645, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.7840203887970143, 'colsample_bytree': 0.7921980207410589, 'gamma': 3.728602221209726}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=21, value=None)
Best hyperparameters:
learning_rate: 0.1776634149900645
max_depth: 4
min_c

**Tuning XGBoost có chọn đặc trưng với optuna**

In [17]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight= 5.300729681724353,
    random_state=42
)
model_xgb.fit(X_train_selected, y_train)


y_pred = model_xgb.predict(X_test_selected)
print_scores( y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_selected)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.95      0.78      0.86     68556
           1       0.54      0.86      0.67     20763

    accuracy                           0.80     89319
   macro avg       0.75      0.82      0.76     89319
weighted avg       0.86      0.80      0.81     89319

Best threshold: 0.6500000000000001, F1: 0.6883854540706404


In [18]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


THRESHOLD = best_thresh
SCALE_POS_WEIGHT =  5.300729681724353 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_selected, y_train)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_selected)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)

# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_train)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
# print("\nClassification report with threshold = 0.75:")
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:23:22,202] A new study created in memory with name: no-name-e35b1f6b-4531-4c38-98f9-518ce59bcd4f
[I 2025-05-04 04:23:26,322] Trial 0 finished with value: 0.6854552191634518 and parameters: {'learning_rate': 0.10041802312710581, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.6264947837858241, 'colsample_bytree': 0.781668797345177, 'gamma': 1.413570042205417}. Best is trial 0 with value: 0.6854552191634518.
[I 2025-05-04 04:23:29,233] Trial 1 finished with value: 0.6903778844340116 and parameters: {'learning_rate': 0.08934066936048565, 'max_depth': 5, 'min_child_weight': 8, 'subsample': 0.907870661679223, 'colsample_bytree': 0.6822118885350956, 'gamma': 2.0683623178701}. Best is trial 1 with value: 0.6903778844340116.
[I 2025-05-04 04:23:33,026] Trial 2 finished with value: 0.6862554843064461 and parameters: {'learning_rate': 0.059175656949122926, 'max_depth': 6, 'min_child_weight': 3, 'subsample': 0.8555285883135912, 'colsample_bytree': 0.9594770073956205, 'gamm

Best trial:
FrozenTrial(number=32, state=1, values=[0.6959124337266974], datetime_start=datetime.datetime(2025, 5, 4, 4, 25, 21, 552490), datetime_complete=datetime.datetime(2025, 5, 4, 4, 25, 24, 445702), params={'learning_rate': 0.1758413568131671, 'max_depth': 5, 'min_child_weight': 7, 'subsample': 0.9205497412592422, 'colsample_bytree': 0.7204123473097305, 'gamma': 2.7149398900835893}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=32, value=None)
Best hyperparameters:
learning_rate: 0.1758413568131671
max_depth: 5
min_

**Đánh giá:**

Lớp mưa:
* Recall = 0.80: Mô hình phát hiện được 80% các trường hợp có mưa → rất tốt trong bối cảnh nhãn lệch (khó học).
* F1-score = 0.70: Mức F1-score cao hơn mô hình mặc định (~0.57) hoặc các cấu hình thủ công trước đó.
* Precision = 0.61: Có một số false positives (dự đoán sai là mưa)

Lớp không mưa: Rất tốt, vẫn giữ precision 0.93 và F1-score 0.89

**Không bắt buộc phải cân bằng dữ liệu khi dùng XGBoost vì đã dùng scale_pos_weight, threshold và tham số bằng optuna) nhưng vẫn nên thử**

=> Chọn đặc trưng hay không thì kết quả cũng tương tự nhau.

# 6. Cân bằng dữ liệu (Oversampling)

## 6.1 SMOTE

In [19]:
# SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE()

X_sm, y_sm = sm.fit_resample(X_train, y_train)
X_sm.shape, y_sm.shape

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_sm)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

In [20]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_sm, y_sm)

y_pred = model_xgb.predict(X_test)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.92      0.88      0.90     68556
           1       0.64      0.74      0.69     20763

    accuracy                           0.84     89319
   macro avg       0.78      0.81      0.79     89319
weighted avg       0.85      0.84      0.85     89319



In [21]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.92      0.87      0.89     68556
           1       0.63      0.74      0.68     20763

    accuracy                           0.84     89319
   macro avg       0.77      0.80      0.79     89319
weighted avg       0.85      0.84      0.84     89319



In [22]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled[top_features], y_sm)

y_pred = model_xgb.predict(X_test_selected)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.92      0.75      0.83     68556
           1       0.49      0.79      0.61     20763

    accuracy                           0.76     89319
   macro avg       0.71      0.77      0.72     89319
weighted avg       0.82      0.76      0.78     89319



In [23]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, min_child_weight=5)
model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.91      0.87      0.89     68556
           1       0.64      0.73      0.68     20763

    accuracy                           0.84     89319
   macro avg       0.77      0.80      0.79     89319
weighted avg       0.85      0.84      0.84     89319



=> Thêm min_child_weight có cải thiện

In [24]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, scale_pos_weight= 5.300729681724353)
model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.95      0.76      0.84     68556
           1       0.52      0.87      0.65     20763

    accuracy                           0.78     89319
   macro avg       0.73      0.81      0.75     89319
weighted avg       0.85      0.78      0.80     89319



=> Thêm scale_pos_weight giảm

In [25]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 5.300729681724353,
    random_state=42,
    min_child_weight=5
    
)

model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores(y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_scaled)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.93      0.84      0.88     68556
           1       0.60      0.80      0.69     20763

    accuracy                           0.83     89319
   macro avg       0.77      0.82      0.79     89319
weighted avg       0.86      0.83      0.84     89319

Best threshold: 0.5500000000000002, F1: 0.6889763779527559


In [26]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  5.300729681724353 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled, y_sm)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_sm)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:27:12,353] A new study created in memory with name: no-name-5b462ffc-1836-4cd2-9cde-af667a4ec0aa
[I 2025-05-04 04:27:17,473] Trial 0 finished with value: 0.6879481551165465 and parameters: {'learning_rate': 0.26463339190886404, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.5662357931171376, 'colsample_bytree': 0.7904642584315256, 'gamma': 4.185743382884682}. Best is trial 0 with value: 0.6879481551165465.
[I 2025-05-04 04:27:28,943] Trial 1 finished with value: 0.689149289536815 and parameters: {'learning_rate': 0.11737568665998574, 'max_depth': 9, 'min_child_weight': 3, 'subsample': 0.8560898290088768, 'colsample_bytree': 0.5282183463814731, 'gamma': 2.0880495406530715}. Best is trial 1 with value: 0.689149289536815.
[I 2025-05-04 04:27:40,486] Trial 2 finished with value: 0.6846741518676047 and parameters: {'learning_rate': 0.018309807889233556, 'max_depth': 8, 'min_child_weight': 8, 'subsample': 0.7546643920379273, 'colsample_bytree': 0.8986236763656644, 'ga

Best trial:
FrozenTrial(number=12, state=1, values=[0.6955440236737145], datetime_start=datetime.datetime(2025, 5, 4, 4, 29, 15, 803283), datetime_complete=datetime.datetime(2025, 5, 4, 4, 29, 23, 863411), params={'learning_rate': 0.07313993550401465, 'max_depth': 7, 'min_child_weight': 8, 'subsample': 0.8946006919806336, 'colsample_bytree': 0.719733928713923, 'gamma': 4.963548791976756}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=12, value=None)
Best hyperparameters:
learning_rate: 0.07313993550401465
max_depth: 7
min_

In [27]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 5.300729681724353,
    random_state=42,
    min_child_weight=5
    
)

model_xgb.fit(X_train_scaled[top_features], y_sm)

y_pred = model_xgb.predict(X_test_selected)
print_scores(y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_selected)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.95      0.74      0.83     68556
           1       0.51      0.86      0.64     20763

    accuracy                           0.77     89319
   macro avg       0.73      0.80      0.74     89319
weighted avg       0.84      0.77      0.79     89319

Best threshold: 0.6500000000000001, F1: 0.6460005309481508


In [28]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  5.300729681724353 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled[top_features], y_sm)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_selected)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled[top_features], y_sm)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_selected)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:31:48,277] A new study created in memory with name: no-name-b9eb576e-2df9-4aa5-b6c7-1d013a9a7582
[I 2025-05-04 04:31:55,397] Trial 0 finished with value: 0.6047882285993009 and parameters: {'learning_rate': 0.21397203711181165, 'max_depth': 10, 'min_child_weight': 4, 'subsample': 0.9892848753170784, 'colsample_bytree': 0.5131404341312547, 'gamma': 1.6883399208890693}. Best is trial 0 with value: 0.6047882285993009.
[I 2025-05-04 04:32:00,321] Trial 1 finished with value: 0.6661021189784253 and parameters: {'learning_rate': 0.019176482323560444, 'max_depth': 4, 'min_child_weight': 9, 'subsample': 0.5762214298474874, 'colsample_bytree': 0.7086829708041282, 'gamma': 2.235584564685942}. Best is trial 1 with value: 0.6661021189784253.
[I 2025-05-04 04:32:04,689] Trial 2 finished with value: 0.6605540612176782 and parameters: {'learning_rate': 0.050600314537826305, 'max_depth': 4, 'min_child_weight': 10, 'subsample': 0.9732200485944968, 'colsample_bytree': 0.8948381351816808

Best trial:
FrozenTrial(number=12, state=1, values=[0.6745512564818509], datetime_start=datetime.datetime(2025, 5, 4, 4, 33, 0, 531996), datetime_complete=datetime.datetime(2025, 5, 4, 4, 33, 7, 270), params={'learning_rate': 0.011745851095688764, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.5169744620782467, 'colsample_bytree': 0.6365706864975532, 'gamma': 4.988231735908286}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=12, value=None)
Best hyperparameters:
learning_rate: 0.011745851095688764
max_depth: 6
min_ch

**Đối với dùng SMOTE:** chỉ scale cao hơn scale xong chọn đặc trưng nhưng không SMOTE cao hơn đã SMOTE.

## 6.2 SMOTE ENN

In [29]:
# SMOTE ENN
from imblearn.combine import SMOTEENN

smenn = SMOTEENN()

X_smenn, y_smenn = smenn.fit_resample(X_train, y_train)
X_smenn.shape, y_sm.shape

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_smenn)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

In [30]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_smenn, y_smenn)

y_pred = model_xgb.predict(X_test)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.92      0.86      0.89     68556
           1       0.63      0.76      0.69     20763

    accuracy                           0.84     89319
   macro avg       0.77      0.81      0.79     89319
weighted avg       0.85      0.84      0.84     89319



In [31]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled, y_smenn)

y_pred = model_xgb.predict(X_test_scaled)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.93      0.85      0.89     68556
           1       0.62      0.77      0.69     20763

    accuracy                           0.84     89319
   macro avg       0.77      0.81      0.79     89319
weighted avg       0.85      0.84      0.84     89319



In [32]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled[top_features], y_smenn)

y_pred = model_xgb.predict(X_test_selected)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.75      0.84     68556
           1       0.51      0.83      0.63     20763

    accuracy                           0.77     89319
   macro avg       0.72      0.79      0.73     89319
weighted avg       0.84      0.77      0.79     89319



=> Không chọn đặc trưng, scale hay không cũng như nhau.

In [33]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, min_child_weight=5)
model_xgb.fit(X_train_scaled, y_smenn)

y_pred = model_xgb.predict(X_test_scaled)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.92      0.87      0.89     68556
           1       0.63      0.76      0.69     20763

    accuracy                           0.84     89319
   macro avg       0.78      0.81      0.79     89319
weighted avg       0.85      0.84      0.85     89319



In [34]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 5.300729681724353,
    random_state=42
)
model_xgb.fit(X_train_scaled, y_smenn)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_scaled)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.94      0.82      0.88     68556
           1       0.58      0.84      0.69     20763

    accuracy                           0.82     89319
   macro avg       0.76      0.83      0.78     89319
weighted avg       0.86      0.82      0.83     89319

Best threshold: 0.6000000000000002, F1: 0.6971541302887844


In [35]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  5.300729681724353

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled, y_smenn)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_smenn)

# Dự đoán với threshold 
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:44:26,717] A new study created in memory with name: no-name-da26e945-bbd2-4759-9da0-5c382a9ac93a
[I 2025-05-04 04:44:33,277] Trial 0 finished with value: 0.6928047861530382 and parameters: {'learning_rate': 0.19754066062073516, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.8255568280724376, 'colsample_bytree': 0.6729765634547814, 'gamma': 2.7724297146775094}. Best is trial 0 with value: 0.6928047861530382.
[I 2025-05-04 04:44:40,105] Trial 1 finished with value: 0.6863910716319326 and parameters: {'learning_rate': 0.27927374840541824, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.6318377768137683, 'colsample_bytree': 0.571836302706731, 'gamma': 2.3259584534725333}. Best is trial 0 with value: 0.6928047861530382.
[I 2025-05-04 04:44:44,290] Trial 2 finished with value: 0.6930149693854566 and parameters: {'learning_rate': 0.2222060901047481, 'max_depth': 3, 'min_child_weight': 7, 'subsample': 0.6135190484313668, 'colsample_bytree': 0.5112287863478933, 'ga

Best trial:
FrozenTrial(number=27, state=1, values=[0.6988443826570511], datetime_start=datetime.datetime(2025, 5, 4, 4, 47, 13, 602027), datetime_complete=datetime.datetime(2025, 5, 4, 4, 47, 19, 885954), params={'learning_rate': 0.13443377943422463, 'max_depth': 6, 'min_child_weight': 6, 'subsample': 0.9362751341208229, 'colsample_bytree': 0.7988673841651286, 'gamma': 0.6389467023496083}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=27, value=None)
Best hyperparameters:
learning_rate: 0.13443377943422463
max_depth: 6
mi

In [36]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 5.300729681724353,
    random_state=42
)
model_xgb.fit(X_train_scaled[top_features], y_smenn)


y_pred = model_xgb.predict(X_test_selected)
print_scores(y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_selected)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.95      0.74      0.84     68556
           1       0.51      0.88      0.64     20763

    accuracy                           0.77     89319
   macro avg       0.73      0.81      0.74     89319
weighted avg       0.85      0.77      0.79     89319

Best threshold: 0.7000000000000002, F1: 0.6646413570933409


In [37]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  5.300729681724353

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled[top_features], y_smenn)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_selected)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled[top_features], y_smenn)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_selected)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:47:45,388] A new study created in memory with name: no-name-5f7d0ee0-d3a4-49c6-8cb9-e52010e2d5b9
[I 2025-05-04 04:47:48,506] Trial 0 finished with value: 0.661296540529573 and parameters: {'learning_rate': 0.16952674463844328, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.5656453833446875, 'colsample_bytree': 0.9610105251593003, 'gamma': 4.187022556760638}. Best is trial 0 with value: 0.661296540529573.
[I 2025-05-04 04:47:52,559] Trial 1 finished with value: 0.6433575078292401 and parameters: {'learning_rate': 0.23828730355541147, 'max_depth': 9, 'min_child_weight': 4, 'subsample': 0.6885683115125845, 'colsample_bytree': 0.5038060093507879, 'gamma': 4.545312499685777}. Best is trial 0 with value: 0.661296540529573.
[I 2025-05-04 04:47:56,736] Trial 2 finished with value: 0.6713526898039059 and parameters: {'learning_rate': 0.05623938745409255, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.6647102838581731, 'colsample_bytree': 0.7526771736276772, 'gamma

Best trial:
FrozenTrial(number=21, state=1, values=[0.6801403508771929], datetime_start=datetime.datetime(2025, 5, 4, 4, 49, 39, 761428), datetime_complete=datetime.datetime(2025, 5, 4, 4, 49, 44, 798035), params={'learning_rate': 0.015927792869107955, 'max_depth': 6, 'min_child_weight': 6, 'subsample': 0.8380601003741719, 'colsample_bytree': 0.6786766947584587, 'gamma': 3.27733769790871}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=21, value=None)
Best hyperparameters:
learning_rate: 0.015927792869107955
max_depth: 6
mi

# 7. Kết luận

Mô hình XGBoost ban đầu (không SMOTE hay SMOTE ENN) là tốt nhất.